In [2]:
from pyspark.sql.functions import *

from pyspark.sql import SparkSession

spark = SparkSession \
    .builder \
    .appName("Python Spark SQL basic example") \
    .config("spark.some.config.option", "some-value") \
    .getOrCreate()

In [7]:
df_emp= spark.read.csv("employees.csv",header=True,inferSchema=True)

In [56]:
df_emp.show(10)
#df_emp.createTempView("employee")

df_manager = df_emp.select(concat_ws(' ',df_emp.FIRST_NAME,df_emp.LAST_NAME).alias("Manager_name"),df_emp.EMPLOYEE_ID)


+-----------+----------+---------+--------+------------+---------+----------+------+--------------+----------+-------------+
|EMPLOYEE_ID|FIRST_NAME|LAST_NAME|   EMAIL|PHONE_NUMBER|HIRE_DATE|    JOB_ID|SALARY|COMMISSION_PCT|MANAGER_ID|DEPARTMENT_ID|
+-----------+----------+---------+--------+------------+---------+----------+------+--------------+----------+-------------+
|        198|    Donald| OConnell|DOCONNEL|650.507.9833|21-JUN-07|  SH_CLERK|  2600|            - |       124|           50|
|        199|   Douglas|    Grant|  DGRANT|650.507.9844|13-JAN-08|  SH_CLERK|  2600|            - |       124|           50|
|        200|  Jennifer|   Whalen| JWHALEN|515.123.4444|17-SEP-03|   AD_ASST|  4400|            - |       101|           10|
|        201|   Michael|Hartstein|MHARTSTE|515.123.5555|17-FEB-04|    MK_MAN| 13000|            - |       100|           20|
|        202|       Pat|      Fay|    PFAY|603.123.6666|17-AUG-05|    MK_REP|  6000|            - |       201|           20|


In [54]:
df_manager.show()

+-----------------+----------+
|     Manager_name|MANAGER_ID|
+-----------------+----------+
|  Donald OConnell|       124|
|    Douglas Grant|       124|
|  Jennifer Whalen|       101|
|Michael Hartstein|       100|
|          Pat Fay|       201|
|     Susan Mavris|       101|
|     Hermann Baer|       101|
|  Shelley Higgins|       101|
|    William Gietz|       205|
|      Steven King|        - |
|    Neena Kochhar|       100|
|      Lex De Haan|       100|
| Alexander Hunold|       102|
|      Bruce Ernst|       103|
|     David Austin|       103|
|  Valli Pataballa|       103|
|    Diana Lorentz|       103|
|  Nancy Greenberg|       101|
|    Daniel Faviet|       108|
|        John Chen|       108|
+-----------------+----------+
only showing top 20 rows



In [57]:
df_join = df_emp\
            .join(df_manager,df_emp.MANAGER_ID==df_manager.EMPLOYEE_ID,'inner')
#.select("emp".EMPLOYEE_ID,"man".MANAGER_ID)

In [60]:
df_join.select(concat_ws(' ',df_join.FIRST_NAME,df_join.LAST_NAME).alias("emp_name"),df_join.Manager_name). show()

+-----------------+-----------------+
|         emp_name|     Manager_name|
+-----------------+-----------------+
|  Donald OConnell|    Kevin Mourgos|
|    Douglas Grant|    Kevin Mourgos|
|  Jennifer Whalen|    Neena Kochhar|
|Michael Hartstein|      Steven King|
|          Pat Fay|Michael Hartstein|
|     Susan Mavris|    Neena Kochhar|
|     Hermann Baer|    Neena Kochhar|
|  Shelley Higgins|    Neena Kochhar|
|    William Gietz|  Shelley Higgins|
|    Neena Kochhar|      Steven King|
|      Lex De Haan|      Steven King|
| Alexander Hunold|      Lex De Haan|
|      Bruce Ernst| Alexander Hunold|
|     David Austin| Alexander Hunold|
|  Valli Pataballa| Alexander Hunold|
|    Diana Lorentz| Alexander Hunold|
|  Nancy Greenberg|    Neena Kochhar|
|    Daniel Faviet|  Nancy Greenberg|
|        John Chen|  Nancy Greenberg|
|   Ismael Sciarra|  Nancy Greenberg|
+-----------------+-----------------+
only showing top 20 rows



In [27]:
spark.sql('''
Select  e.employee_id,concat(e.FIRST_NAME,' ',e.LAST_NAME) as emp_name , concat(m.FIRST_NAME,' ',m.LAST_NAME) as Manager_name,m.manager_id
from employee e inner join 
employee m 
on e.manager_id = m.employee_id''').show()


+-----------+-----------------+-----------------+----------+
|employee_id|         emp_name|     Manager_name|manager_id|
+-----------+-----------------+-----------------+----------+
|        198|  Donald OConnell|    Kevin Mourgos|       100|
|        199|    Douglas Grant|    Kevin Mourgos|       100|
|        200|  Jennifer Whalen|    Neena Kochhar|       100|
|        201|Michael Hartstein|      Steven King|        - |
|        202|          Pat Fay|Michael Hartstein|       100|
|        203|     Susan Mavris|    Neena Kochhar|       100|
|        204|     Hermann Baer|    Neena Kochhar|       100|
|        205|  Shelley Higgins|    Neena Kochhar|       100|
|        206|    William Gietz|  Shelley Higgins|       101|
|        101|    Neena Kochhar|      Steven King|        - |
|        102|      Lex De Haan|      Steven King|        - |
|        103| Alexander Hunold|      Lex De Haan|       100|
|        104|      Bruce Ernst| Alexander Hunold|       102|
|        105|     David 